In [185]:
import pandas as pd
from gurobipy import *
from pyomo.environ import *

In [340]:
xls = pd.ExcelFile('miniset_2.xlsx', engine='openpyxl')
nombres= xls.sheet_names
df1 = pd.read_excel(xls, nombres[0])
df2 = pd.read_excel(xls, nombres[1])
df3= pd.read_excel(xls, nombres[2])
df4= pd.read_excel(xls, nombres[3])
df5= pd.read_excel(xls, nombres[4])

In [341]:
barcos_sheet = df1.to_dict("index")
compatibilidad_sheet = df2.to_dict()
cargos_sheet = df3.to_dict("index")
costos_transporte_sheet = df4.to_dict("index")
costos_transporte_dict = {(costos_transporte_sheet[i]['ID_Barco'],costos_transporte_sheet[i]['Puerto_Origen'],costos_transporte_sheet[i]['Puerto_Destino']) : {'Tiempo_Viaje_hrs':costos_transporte_sheet[i]['Tiempo_Viaje_hrs'], 'Costo_Viaje_libras':costos_transporte_sheet[i]['Costo_Viaje_libras']} for i in costos_transporte_sheet.keys()}
costos_tiempo_sheet = df5.to_dict("index")
costos_tiempo_dict = {(costos_tiempo_sheet[i]['ID_Barco'],costos_tiempo_sheet[i]['ID_Cargo']) : {'Tiempo_Origen_hrs':costos_tiempo_sheet[i]['Tiempo_Origen_hrs'], 'Costos_Origen_hrs':costos_tiempo_sheet[i]['Costos_Origen_hrs'], 'Tiempo_Destino_hrs':costos_tiempo_sheet[i]['Tiempo_Destino_hrs'], 'Costos_Destino_hrs':costos_tiempo_sheet[i]['Costos_Destino_hrs']} for i in costos_tiempo_sheet.keys()}

In [342]:
barcos_sheet

{0: {'ID_Barco': 1,
  'ID_Puerto ': 3,
  'Tiempo_Inicio': 69,
  'Capacidad': 16500,
  'Nombre_Barco': 'ShortDryBulk16_1',
  'Nombre_Puerto': 'ROTTERDAM'}}

In [343]:
n_cargos = len(cargos_sheet)
n_cargos

2

In [344]:
V = [1+i for i in barcos_sheet.keys()] ## conjunto barcos
n_barcos = len(V)
print(V)

[1]


In [345]:
Kv= {} # conjunto capacidades de barcos
for i in barcos_sheet.keys():
    Kv[barcos_sheet[i]['ID_Barco']] =  barcos_sheet[i]['Capacidad']
Kv

{1: 16500}

In [346]:
# N^P
nodos_carga= [cargos_sheet[i]['ID_Cargo'] for i in cargos_sheet.keys()]
print(nodos_carga)

#N^D
nodos_descarga= [i+len(nodos_carga) for i in range(1,len(nodos_carga)+1)] ##conjunto nodos de descarga
print(nodos_descarga)

[1, 2]
[3, 4]


In [347]:
nodos = {}
for i in range(n_cargos):
    nodos[cargos_sheet[i]['ID_Cargo']] = {'ID_Cargo': cargos_sheet[i]['ID_Cargo'],
                                          'ID_Puerto': cargos_sheet[i]['ID_Puerto_Origen'],
                                          'Tamano': cargos_sheet[i]['Tamano'],
                                          'Costo_SPOT': cargos_sheet[i]['Costo_SPOT'],
                                          'LT': cargos_sheet[i]['LT_Carga'],
                                          'RT': cargos_sheet[i]['RT_Carga']}
    
    nodos[cargos_sheet[i]['ID_Cargo']+n_cargos] = {'ID_Cargo': cargos_sheet[i]['ID_Cargo'],
                                          'ID_Puerto': cargos_sheet[i]['ID_Puerto_Destino'],
                                          'Tamano': cargos_sheet[i]['Tamano'],
                                          'Costo_SPOT': cargos_sheet[i]['Costo_SPOT'],
                                          'LT': cargos_sheet[i]['LT_Descarga'],
                                          'RT': cargos_sheet[i]['RT_Descarga']}
o = {}
d = {}
for v in V:
    o[v] = n_cargos*2+v
    nodos[n_cargos*2+v] = {'Costo_SPOT': 0,
                           'ID_Cargo': 0,
                           'ID_Puerto': barcos_sheet[v-1]['ID_Puerto '],
                           'LT': barcos_sheet[v-1]['Tiempo_Inicio'],
                           'RT': barcos_sheet[v-1]['Tiempo_Inicio'],
                           'Tamano': 0}
    d[v] = n_cargos*2+n_barcos+v
    nodos[n_cargos*2+n_barcos+v] = {'Costo_SPOT': 0,
                                    'ID_Cargo': 0,
                                    'ID_Puerto': 0,
                                    'LT': 777,
                                    'RT': 777,
                                    'Tamano': 0}
print(o) # nodos de origen
print(d) # nodos de destino
print(nodos) # todos los nodos con su información

{1: 5}
{1: 6}
{1: {'ID_Cargo': 1, 'ID_Puerto': 25, 'Tamano': 2368, 'Costo_SPOT': 354894, 'LT': 383, 'RT': 455}, 3: {'ID_Cargo': 1, 'ID_Puerto': 13, 'Tamano': 2368, 'Costo_SPOT': 354894, 'LT': 383, 'RT': 777}, 2: {'ID_Cargo': 2, 'ID_Puerto': 9, 'Tamano': 6919, 'Costo_SPOT': 536901, 'LT': 149, 'RT': 221}, 4: {'ID_Cargo': 2, 'ID_Puerto': 6, 'Tamano': 6919, 'Costo_SPOT': 536901, 'LT': 149, 'RT': 588}, 5: {'Costo_SPOT': 0, 'ID_Cargo': 0, 'ID_Puerto': 3, 'LT': 69, 'RT': 69, 'Tamano': 0}, 6: {'Costo_SPOT': 0, 'ID_Cargo': 0, 'ID_Puerto': 0, 'LT': 777, 'RT': 777, 'Tamano': 0}}


In [348]:
compatibilidad = {}
for i in V:
    compatibilidad[i] = [int(j) for j in list(df2.loc[i-1])[1:] if j>0]
compatibilidad

Nv = {}
for i in V:
    Nv[i] = []
    for j in nodos.keys():
        if nodos[j]["ID_Cargo"] in compatibilidad[i] or (j == i+n_cargos*2 or j==i+n_cargos*2+len(V)): 
            Nv[i].append(j)
Nv

{1: [1, 3, 5, 6]}

In [349]:
"""import itertools
Av = {}
for v in V:
    Av[v] = list(itertools.permutations(Nv[v],2))
"""
Av={}
indices = []
for v in V:
    Av[v] = [] 
    for i in Nv[v]:
        for j in Nv[v]:
                Av[v].append((i,j))
                indices.append((i,j,v))
indices = tuplelist(indices)
print(Av[1])

[(1, 1), (1, 3), (1, 5), (1, 6), (3, 1), (3, 3), (3, 5), (3, 6), (5, 1), (5, 3), (5, 5), (5, 6), (6, 1), (6, 3), (6, 5), (6, 6)]


In [350]:
NP_v = {}
for v in V:
    NP_v[v] = [i for i in nodos_carga if i in Nv[v]]

ND_v = {}
for v in V:
    ND_v[v] = [i for i in nodos_descarga if i in Nv[v]]

print(NP_v)
print(ND_v)

{1: [1]}
{1: [3]}


In [351]:
LT_i = {i:nodos[i]["LT"] for i in nodos.keys()} #lower bounds ventana de tiempo por nodo
RT_i = {i:nodos[i]["RT"] for i in nodos.keys()} #upper bounds ventana de tiempo por nodo

print(LT_i)
print(RT_i)

{1: 383, 3: 383, 2: 149, 4: 149, 5: 69, 6: 777}
{1: 455, 3: 777, 2: 221, 4: 588, 5: 69, 6: 777}


In [352]:
T_ijv={} # tiempo de entre i y j (tiempo de viaje + tiempo de operación en i)
for i in nodos.keys():
    T_ijv[i] = {} 
    for j in nodos.keys():
        T_ijv[i][j] = {} 
        for v in V:
            cargo_i= nodos[i]["ID_Cargo"]
            cargo_j = nodos[j]["ID_Cargo"]
            if i>n_cargos*2+len(V) or j>n_cargos*2+len(V):
                T_ijv[i][j][v] = 0
            else: 
                puerto_origen = nodos[i]["ID_Puerto"]
                puerto_destino = nodos[j]["ID_Puerto"]
                T_ijv[i][j][v] = costos_transporte_dict[v,puerto_origen,puerto_destino]['Tiempo_Viaje_hrs']
                if 0<i<=n_cargos and (i,j) in Av[v]: 
                    T_ijv[i][j][v] += costos_tiempo_dict[v,cargo_i]['Tiempo_Origen_hrs']
                if n_cargos*2>=i>n_cargos and (i,j) in Av[v]:
                    T_ijv[i][j][v] += costos_tiempo_dict[v,cargo_i]['Tiempo_Destino_hrs']
                
                # se les asigna tiempo de operación igual a 0 a los (i,j) que no pertenecen a Av
                # los (i,j) que no pertenecen a Av aún tienen el tiempo entre el puerto de i y j

In [353]:
C_ijv={} # costo de entre i y j (costo de viaje + costo de operación en i)
for i in nodos.keys():
    C_ijv[i] = {} 
    for j in nodos.keys():
        C_ijv[i][j] = {} 
        for v in V:
            cargo_i= nodos[i]["ID_Cargo"]
            cargo_j = nodos[j]["ID_Cargo"]
            if i>n_cargos*2+len(V) or j>n_cargos*2+len(V):
                C_ijv[i][j][v] = 0
            else: 
                puerto_origen = nodos[i]["ID_Puerto"]
                puerto_destino = nodos[j]["ID_Puerto"]
                C_ijv[i][j][v] = costos_transporte_dict[v,puerto_origen,puerto_destino]['Costo_Viaje_libras']
                if i==11 and j==2 and i==1:
                    print(C_ijv[i][j][v])
                if 0<i<=n_cargos and (i,j) in Av[v]:
                    C_ijv[i][j][v] += costos_tiempo_dict[v,cargo_i]['Costos_Origen_hrs']

                if n_cargos*2>=i>n_cargos and (i,j) in Av[v]:
                    C_ijv[i][j][v] += costos_tiempo_dict[v,cargo_i]['Costos_Destino_hrs']

In [354]:
Qj={i:nodos[i]["Tamano"] for i in nodos.keys()}
print("Qj: ",Qj)

print("\n")

Cs = {cargos_sheet[i]['ID_Cargo']:cargos_sheet[i]['Costo_SPOT'] for i in cargos_sheet.keys()}
print("Cs", Cs)

Qj:  {1: 2368, 3: 2368, 2: 6919, 4: 6919, 5: 0, 6: 0}


Cs {1: 354894, 2: 536901}


In [355]:
nodos_index = [i for i in nodos.keys()]

In [356]:
Nv[1]

[1, 3, 5, 6]

In [357]:
def obj_rule(model):
    return sum(C_ijv[i][j][v]*model.x[i,j,v] for (i,j) in Av[v] for v in V)+sum(Cs[i]*model.y[i] for i in nodos_carga)

def const_1(model, i):
    return sum(model.x[i,j,v] for j in Nv[v] for v in V) + model.y[i] == 1

def const_2(model, v):
    return sum(model.x[o[v],j,v] for j in Nv[v]) == 1

def const_3(model, i, v):
    if i in Nv[v]:
        if i!=o[v] and i!=d[v]:
            return sum(model.x[i,j,v] for j in Nv[v])-sum(model.x[j,i,v] for j in Nv[v])==0
    return Constraint.Skip

def const_4(model, v):
    return sum(model.x[j,d[v],v] for j in Nv[v]) == 1

def const_5(model, i, j, v):
    if j in NP_v[v]:
        if (i,j) in Av[v]:
            return model.l[i,v]+Qj[j]-model.l[j,v]<=Kv[v]*(1-model.x[i,j,v])
    return Constraint.Skip

def const_6(model, i, j, v):
    if j in NP_v[v]:
        if (i,n_cargos+j) in Av[v]:
            return model.l[i,v]-Qj[j]-model.l[n_cargos+j,v]<=Kv[v]*(1-model.x[i,n_cargos+j,v])
    return Constraint.Skip

def const_7_ub(model, i, v):
    if i in NP_v[v]:
        return model.l[i,v]<=Kv[v]
    return Constraint.Skip

def const_7_lb(model, i, v):
    if i in NP_v[v]:
        return 0<=model.l[i,v]
    return Constraint.Skip
def const_8(model, i, j, v):
    if (i,j) in Av[v]:
        return model.t[i,v]+T_ijv[i][j][v]-model.t[j,v]<=(RT_i[i]+T_ijv[i][j][v])*(1-model.x[i,j,v])
    return Constraint.Skip

def const_9(model, i, v):
    if i in NP_v[v]:
        return sum(model.x[i,j,v] for j in Nv[v])-sum(model.x[n_cargos+i,j,v] for j in Nv[v])==0
    return Constraint.Skip

def const_10(model, i, v):
    if i in NP_v[v]:
        return model.t[i,v]+T_ijv[i][n_cargos+i][v]-model.t[n_cargos+i,v]<=0
    return Constraint.Skip
    
def const_11_ub(model, i, v):
    if i in Nv[v]:
        return model.t[i,v]<=RT_i[i]
    return Constraint.Skip

def const_11_lb(model, i, v):
    if i in Nv[v]:
        return LT_i[i]<=model.t[i,v]
    return Constraint.Skip


"""
"""

'\n'

In [358]:
model_1 = ConcreteModel()

In [359]:
model_1.x = Var(nodos_index, nodos_index, V, within=Binary)
model_1.y = Var(nodos_carga, within=Binary)
model_1.t = Var(nodos_index,V)
model_1.l = Var(nodos_index,V)

In [360]:
model_1.obj = Objective(rule=obj_rule)
model_1.const_1 = Constraint(nodos_carga, rule=const_1)
model_1.const_2 = Constraint(V, rule=const_2)
model_1.const_3 = Constraint(nodos_index, V, rule=const_3)
model_1.const_4 = Constraint(V, rule=const_4)
model_1.const_5 = Constraint(nodos_index, nodos_index, V, rule=const_5)
model_1.const_6 = Constraint(nodos_index, nodos_index, V, rule=const_6)
model_1.const_7_ub = Constraint(nodos_index, V, rule=const_7_ub)
model_1.const_7_lb = Constraint(nodos_index, V, rule=const_7_lb)
model_1.const_8 = Constraint(nodos_index, nodos_index, V, rule=const_8)
model_1.const_9 = Constraint(nodos_index, V, rule=const_9)
model_1.const_10 = Constraint(nodos_index, V, rule=const_10)
model_1.const_11_ub = Constraint(nodos_index, V, rule=const_11_ub)
model_1.const_11_lb = Constraint(nodos_index, V, rule=const_11_lb)
"""
"""

'\n'

In [361]:
solver_1 = SolverFactory('gurobi')
solver_1.solve(model_1)['Solver'][0]['Message']

'Model was solved to optimality (subject to tolerances), and an optimal solution is available.'

In [362]:
Kv[1]

16500

In [363]:
model_1.x.extract_values()

{(1, 1, 1): 0.0,
 (1, 3, 1): 1.0,
 (1, 2, 1): None,
 (1, 4, 1): None,
 (1, 5, 1): 0.0,
 (1, 6, 1): 0.0,
 (3, 1, 1): 0.0,
 (3, 3, 1): 0.0,
 (3, 2, 1): None,
 (3, 4, 1): None,
 (3, 5, 1): 0.0,
 (3, 6, 1): 1.0,
 (2, 1, 1): 1.0,
 (2, 3, 1): 0.0,
 (2, 2, 1): None,
 (2, 4, 1): None,
 (2, 5, 1): 0.0,
 (2, 6, 1): 0.0,
 (4, 1, 1): None,
 (4, 3, 1): None,
 (4, 2, 1): None,
 (4, 4, 1): None,
 (4, 5, 1): None,
 (4, 6, 1): None,
 (5, 1, 1): 1.0,
 (5, 3, 1): 0.0,
 (5, 2, 1): None,
 (5, 4, 1): None,
 (5, 5, 1): 0.0,
 (5, 6, 1): 0.0,
 (6, 1, 1): 0.0,
 (6, 3, 1): 0.0,
 (6, 2, 1): None,
 (6, 4, 1): None,
 (6, 5, 1): 0.0,
 (6, 6, 1): 0.0}

In [364]:
model_1.y.pprint()

y : Size=2, Index=y_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   0.0 :     1 : False : False : Binary
      2 :     0 :   0.0 :     1 : False : False : Binary


In [365]:
model_1.t.pprint()

t : Size=6, Index=t_index
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 1) :  None : 383.0 :  None : False : False :  Reals
    (2, 1) :  None :  None :  None : False :  True :  Reals
    (3, 1) :  None : 777.0 :  None : False : False :  Reals
    (4, 1) :  None :  None :  None : False :  True :  Reals
    (5, 1) :  None :  69.0 :  None : False : False :  Reals
    (6, 1) :  None : 777.0 :  None : False : False :  Reals


In [366]:
model_1.l.pprint()

l : Size=6, Index=l_index
    Key    : Lower : Value   : Upper : Fixed : Stale : Domain
    (1, 1) :  None :     0.0 :  None : False : False :  Reals
    (2, 1) :  None :    None :  None : False :  True :  Reals
    (3, 1) :  None : 14132.0 :  None : False : False :  Reals
    (4, 1) :  None :    None :  None : False :  True :  Reals
    (5, 1) :  None : -2368.0 :  None : False : False :  Reals
    (6, 1) :  None : 14132.0 :  None : False : False :  Reals


In [367]:
model_1.obj.expr()

103958.0

In [368]:
C_ijv[3][1][1]+C_ijv[1][2][1]+C_ijv[1][4][1]

180705